<a href="https://colab.research.google.com/github/MR-Toufigh/Basics-of-intelligent-systems-fall-2024/blob/main/The_second_mini_project_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings

# Suppress all warnings to keep the output clean
warnings.filterwarnings("ignore")

# Upgrade the kagglehub library silently
!pip install --upgrade kagglehub -q

import kagglehub
import shutil
import os

# Destination path where the files will be moved
destination_path = "/content"

# Ensure the destination directory exists, create it if necessary
os.makedirs(destination_path, exist_ok=True)

# Download files from the Kaggle dataset
try:
    path = kagglehub.dataset_download("aslkuscu/telecust1000t")
    print(f"Dataset downloaded to temporary path: {path}")

    # Move all files and folders from the temporary path to the destination path
    for item in os.listdir(path):
        s = os.path.join(path, item)  # Source path of the item
        d = os.path.join(destination_path, item)  # Destination path of the item

        if os.path.isdir(s):
            # If the item is a directory, use copytree to copy it
            shutil.copytree(s, d, dirs_exist_ok=True)
        else:
            # If the item is a single file, use copy2 to copy it
            shutil.copy2(s, d)

    print(f"All files and folders moved to: {destination_path}")

except Exception as e:
    # Handle any errors during the process
    print(f"An error occurred: {e}")


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
file_path = '/content/teleCust1000t.csv'
data = pd.read_csv(file_path)

# Analyze the dataset
print("Dataset Information:")
data.info()
print("\nDataset Description:")
print(data.describe())

# Visualize the dataset
# Plot histograms of all features
print("\nHistograms of all features:")
data.hist(bins=20, figsize=(15, 10))
plt.show()

# Correlation heatmap
print("\nCorrelation Heatmap:")
correlation_matrix = data.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

# Find the two most correlated features with the target column 'custcat'
if 'custcat' in data.columns:
    target_corr = correlation_matrix['custcat'].sort_values(ascending=False)
    most_correlated = target_corr.index[1:3]  # Exclude the target itself
    print("\nMost correlated features with 'custcat':", most_correlated.tolist())
else:
    print("Target column 'custcat' not found in the dataset.")
    most_correlated = None

# Visualize histograms of the most correlated features if available
if most_correlated is not None:
    for feature in most_correlated:
        plt.figure(figsize=(8, 6))
        sns.histplot(data[feature], kde=True, bins=20)
        plt.title(f"Distribution of {feature}")
        plt.xlabel(feature)
        plt.ylabel("Frequency")
        plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
file_path = '/content/teleCust1000t.csv'
data = pd.read_csv(file_path)

# Apply true one-hot encoding directly in the same columns for features with less than 100 unique classes
def true_one_hot_encode_inplace(df, columns):
    for col in columns:
        unique_vals = df[col].unique()
        unique_count = len(unique_vals)
        value_map = {val: [1 if i == idx else 0 for i in range(unique_count)] for idx, val in enumerate(sorted(unique_vals))}
        df[col] = df[col].map(value_map).apply(lambda x: sum([bit * (2 ** idx) for idx, bit in enumerate(x)]))
    return df

categorical_columns = [col for col in data.columns if data[col].nunique() < 100]
data = true_one_hot_encode_inplace(data, categorical_columns)

# Analyze the dataset
print("Dataset Information:")
data.info()
print("\nDataset Description:")
print(data.describe())

# Visualize the dataset
# Plot histograms of all features
print("\nHistograms of all features:")
data.hist(bins=20, figsize=(15, 10))
plt.show()

# Correlation heatmap
print("\nCorrelation Heatmap:")
correlation_matrix = data.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

# Find the two most correlated features with the target column 'custcat'
if 'custcat' in data.columns:
    target_corr = correlation_matrix['custcat'].sort_values(ascending=False)
    most_correlated = target_corr.index[1:3]  # Exclude the target itself
    print("\nMost correlated features with 'custcat':", most_correlated.tolist())
else:
    print("Target column 'custcat' not found in the dataset.")
    most_correlated = None

# Visualize histograms of the most correlated features if available
if most_correlated is not None:
    for feature in most_correlated:
        plt.figure(figsize=(8, 6))
        sns.histplot(data[feature], kde=True, bins=20)
        plt.title(f"Distribution of {feature}")
        plt.xlabel(feature)
        plt.ylabel("Frequency")
        plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler



# Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
data[data.columns] = scaler.fit_transform(data[data.columns])

# Split the data into train, validation, and test sets
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Output information about the splits
print("Train Set Size:", train_data.shape)
print("Validation Set Size:", validation_data.shape)
print("Test Set Size:", test_data.shape)

# Save the splits to separate files if needed
train_data.to_csv('/content/train_data.csv', index=False)
validation_data.to_csv('/content/validation_data.csv', index=False)
test_data.to_csv('/content/test_data.csv', index=False)


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score

#---------------------------------------------------------------------
# 0) خواندن داده و آماده‌سازی (در صورت نیاز مسیر فایل را اصلاح کنید)
#---------------------------------------------------------------------
df = pd.read_csv("teleCust1000t.csv")  # مسیر فایل CSV خود را قرار دهید

X = df.drop(['custcat'], axis=1).values
y = df['custcat'].values
y_categorical = to_categorical(y - 1)  # چون مقدار کلاس‌ها از 1 تا 4 است

# تقسیم به train/val/test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_categorical, test_size=0.3, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

# نرمال‌سازی ویژگی‌ها با MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

# فقط برای نمایش شکل داده
print("Train shape:", X_train.shape, " Val shape:", X_val.shape, " Test shape:", X_test.shape)

#---------------------------------------------------
# توابع کمکی برای ساخت مدل‌های مختلف
#---------------------------------------------------

def build_model_single_hidden(n_neurons, use_batchnorm=False):
    """
    ساخت یک شبکه‌ی عصبی با تنها یک لایه‌ی مخفی
    بهینه‌ساز: SGD
    پارامتر use_batchnorm تعیین می‌کند که از BatchNormalization استفاده بشود یا خیر.
    """
    model = Sequential()

    # لایه‌ی مخفی
    model.add(Dense(n_neurons, input_dim=X_train.shape[1], kernel_initializer='he_uniform'))
    if use_batchnorm:
        model.add(BatchNormalization())  # لایه‌ی نرمال‌سازی دسته‌ای
    model.add(Activation('relu'))

    # لایه‌ی خروجی (4 کلاس)
    model.add(Dense(4, activation='softmax'))

    # کامپایل با بهینه‌ساز SGD
    model.compile(optimizer=SGD(learning_rate=0.01),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def build_model_two_hidden(n1, n2, use_batchnorm=False):
    """
    ساخت یک شبکه‌ی عصبی با دو لایه‌ی مخفی
    بهینه‌ساز: SGD
    پارامترهای n1, n2 برای تعداد نورون‌های لایه‌های مخفی است.
    پارامتر use_batchnorm تعیین می‌کند که از BatchNormalization در هر لایه‌ی مخفی استفاده بشود یا خیر.
    """
    model = Sequential()

    # لایه‌ی مخفی اول
    model.add(Dense(n1, input_dim=X_train.shape[1], kernel_initializer='he_uniform'))
    if use_batchnorm:
        model.add(BatchNormalization())
    model.add(Activation('relu'))

    # لایه‌ی مخفی دوم
    model.add(Dense(n2, kernel_initializer='he_uniform'))
    if use_batchnorm:
        model.add(BatchNormalization())
    model.add(Activation('relu'))

    # لایه‌ی خروجی
    model.add(Dense(4, activation='softmax'))

    # کامپایل با بهینه‌ساز SGD
    model.compile(optimizer=SGD(learning_rate=0.01),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

#----------------------------------------------------------
# توابع برای نمایش ماتریس سردرگمی و محاسبه دقت هر کلاس
#----------------------------------------------------------

def display_confusion_matrix_with_precision(y_true, y_pred, class_names):
    """
    Displays a confusion matrix with a precision score for each class
    and highlights the confusion matrix in green.

    Parameters:
        y_true (array-like): True labels
        y_pred (array-like): Predicted labels
        class_names (list): Names of the classes

    """
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Compute precision for each class
    precision_per_class = precision_score(y_true, y_pred, average=None)

    # Plot the confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    # Print precision for each class
    for i, class_name in enumerate(class_names):
        print(f"Precision for {class_name}: {precision_per_class[i]:.2f}")

#----------------------------------------
# 1) مدل اول: یک لایه مخفی - حلقه روی n
#----------------------------------------

neuron_options = [8, 16, 32, 64, 128, 256, 512, 1024]  # به عنوان نمونه
additional_numbers = list(range(85, 585, 100))
neuron_options.extend(additional_numbers)
neuron_options = list(set(neuron_options))  # حذف مقادیر تکراری
neuron_options.sort()  # مرتب‌سازی دوباره
print("\n======================== MODEL 1 (One Hidden Layer) ========================")
all_model_1_no_bn = []  # To save results for graph
all_model_1_bn = []     # To save results for graph

for n in neuron_options:
    print(f"\n>> Single hidden layer with {n} neurons")
    model1 = build_model_single_hidden(n_neurons=n, use_batchnorm=False)

    # آموزش مدل:
    history1 = model1.fit(
        X_train_scaled, y_train,
        validation_data=(X_val_scaled, y_val),
        epochs=100, batch_size=32, verbose=0
    )

    # دقت پایانی روی داده‌ی train و val
    train_acc = history1.history['accuracy'][-1]
    val_acc   = history1.history['val_accuracy'][-1]
    print(f"    Final Train Accuracy: {train_acc*100:.2f}%   |   Validation Accuracy: {val_acc*100:.2f}%")

    # ذخیره‌ی نتایج برای رسم نمودار
    all_model_1_no_bn.append((n, train_acc, val_acc))

    # پیش‌بینی داده تست و نمایش ماتریس سردرگمی
    y_pred = np.argmax(model1.predict(X_test_scaled), axis=1)
    y_true = np.argmax(y_test, axis=1)
    display_confusion_matrix_with_precision(y_true, y_pred, ['Class 0', 'Class 1', 'Class 2', 'Class 3'])

#---------------------------------------
# 2) مدل دوم: دو لایه مخفی - حلقه تو در تو
#---------------------------------------

print("\n======================== MODEL 2 (Two Hidden Layers) ========================")

neuron_options_layer1 = [8, 16, 32, 64, 100, 128, 200, 256, 300, 400, 512, 1024]
neuron_options_layer2 = [8, 16, 32, 64, 100, 128, 200, 256]

all_model_2_no_bn = []  # To save results for graph
all_model_2_bn = []     # To save results for graph

for n1 in neuron_options_layer1:
    for n2 in neuron_options_layer2:
        print(f"\n>> Two hidden layers with {n1} and {n2} neurons")
        model2 = build_model_two_hidden(n1=n1, n2=n2, use_batchnorm=False)

        # آموزش مدل:
        history2 = model2.fit(
            X_train_scaled, y_train,
            validation_data=(X_val_scaled, y_val),
            epochs=100, batch_size=32, verbose=0
        )

        train_acc = history2.history['accuracy'][-1]
        val_acc   = history2.history['val_accuracy'][-1]
        print(f"    Final Train Accuracy: {train_acc*100:.2f}%   |   Validation Accuracy: {val_acc*100:.2f}%")

        # ذخیره‌ی نتایج برای رسم نمودار
        all_model_2_no_bn.append(((n1, n2), train_acc, val_acc))

        # پیش‌بینی داده تست و نمایش ماتریس سردرگمی
        y_pred = np.argmax(model2.predict(X_test_scaled), axis=1)
        y_true = np.argmax(y_test, axis=1)
        display_confusion_matrix_with_precision(y_true, y_pred, ['Class 0', 'Class 1', 'Class 2', 'Class 3'])

#--------------------------------------------------------------------------------
# 3) یک حلقه‌ی دیگر برای بررسی اثر لایه‌ی Batch Normalization روی همان مدل‌ها
#--------------------------------------------------------------------------------

print("\n======================== MODEL 1 with BatchNorm (One Hidden Layer) ========================")
for n in neuron_options:
    print(f"\n>> Single hidden layer with {n} neurons + BatchNorm")
    model1_bn = build_model_single_hidden(n_neurons=n, use_batchnorm=True)

    history1_bn = model1_bn.fit(
        X_train_scaled, y_train,
        validation_data=(X_val_scaled, y_val),
        epochs=100, batch_size=32, verbose=0
    )
    train_acc = history1_bn.history['accuracy'][-1]
    val_acc   = history1_bn.history['val_accuracy'][-1]
    print(f"    Final Train Accuracy: {train_acc*100:.2f}%   |   Validation Accuracy: {val_acc*100:.2f}%")

    # ذخیره‌ی نتایج برای رسم نمودار
    all_model_1_bn.append((n, train_acc, val_acc))

    # پیش‌بینی داده تست و نمایش ماتریس سردرگمی
    y_pred = np.argmax(model1_bn.predict(X_test_scaled), axis=1)
    y_true = np.argmax(y_test, axis=1)
    display_confusion_matrix_with_precision(y_true, y_pred, ['Class 0', 'Class 1', 'Class 2', 'Class 3'])

print("\n======================== MODEL 2 with BatchNorm (Two Hidden Layers) ========================")
for n1 in neuron_options_layer1:
    for n2 in neuron_options_layer2:
        print(f"\n>> Two hidden layers with {n1} and {n2} neurons + BatchNorm")
        model2_bn = build_model_two_hidden(n1=n1, n2=n2, use_batchnorm=True)

        history2_bn = model2_bn.fit(
            X_train_scaled, y_train,
            validation_data=(X_val_scaled, y_val),
            epochs=100, batch_size=32, verbose=0
        )
        train_acc = history2_bn.history['accuracy'][-1]
        val_acc   = history2_bn.history['val_accuracy'][-1]
        print(f"    Final Train Accuracy: {train_acc*100:.2f}%   |   Validation Accuracy: {val_acc*100:.2f}%")

        # ذخیره‌ی نتایج برای رسم نمودار
        all_model_2_bn.append(((n1, n2), train_acc, val_acc))

        # پیش‌بینی داده تست و نمایش ماتریس سردرگمی
        y_pred = np.argmax(model2_bn.predict(X_test_scaled), axis=1)
        y_true = np.argmax(y_test, axis=1)
        display_confusion_matrix_with_precision(y_true, y_pred, ['Class 0', 'Class 1', 'Class 2', 'Class 3'])

print("\n==== All experiments finished. ====")

#-----------------------------------------------
# Plotting the results for comparison
#-----------------------------------------------

def plot_results(all_results, title):
    neurons = [x[0] for x in all_results]
    train_acc = [x[1] for x in all_results]
    val_acc = [x[2] for x in all_results]

    plt.figure(figsize=(10, 6))
    plt.plot(neurons, train_acc, label='Train Accuracy')
    plt.plot(neurons, val_acc, label='Validation Accuracy')
    plt.xlabel('Number of Neurons')
    plt.ylabel('Accuracy')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

plot_results(all_model_1_no_bn, 'Model 1 (One Hidden Layer) Without BatchNorm')
plot_results(all_model_1_bn, 'Model 1 (One Hidden Layer) With BatchNorm')

# Plotting results for Model 2

def plot_results_model_2(all_results, title):
    neurons_config = [f"{x[0][0]}-{x[0][1]}" for x in all_results]
    train_acc = [x[1] for x in all_results]
    val_acc = [x[2] for x in all_results]

    plt.figure(figsize=(12, 8))
    plt.plot(neurons_config, train_acc, label='Train Accuracy')
    plt.plot(neurons_config, val_acc, label='Validation Accuracy')
    plt.xlabel('Neurons Configuration (Layer1-Layer2)')
    plt.ylabel('Accuracy')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=90)
    plt.show()

plot_results_model_2(all_model_2_no_bn, 'Model 2 (Two Hidden Layers) Without BatchNorm')
plot_results_model_2(all_model_2_bn, 'Model 2 (Two Hidden Layers) With BatchNorm')
